In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt



postulantesEducacion = pd.read_csv('resources/tar/fiuba_1_postulantes_educacion.csv')
postulantesGenero = pd.read_csv('resources/tar/fiuba_2_postulantes_genero_y_edad.csv')
vistas = pd.read_csv('resources/tar/fiuba_3_vistas.csv')
postulaciones = pd.read_csv('resources/tar/fiuba_4_postulaciones.csv')
avisosOnline = pd.read_csv('resources/tar/fiuba_5_avisos_online.csv')
avisosDetalle = pd.read_csv('resources/tar/fiuba_6_avisos_detalle.csv')

testingSet = pd.read_csv('resources/test_final_100k.csv')

In [2]:
testingSet.head()

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
1,1,739260,6v1xdL
2,2,739260,ezRKm9
3,3,758580,1Q35ej
4,4,758580,EAN4J6


# Importamos librerias ML

# Transformamos Categorias en Numerical

## PostulantesGenero

In [2]:
#primera estimacion al set de prueba
#primeraPrueba = datosForTesting.loc[:, ['lat', 'lon', 'surface_total_in_m2', 'surface_covered_in_m2']]
#RFRModel = RandomForestRegressor(n_estimators=100, max_features='sqrt')

postulantesGenero['fechanacimiento'] = pd.to_datetime(postulantesGenero['fechanacimiento'], format='%Y-%m-%d', errors='coerce')
postulantesGenero = postulantesGenero.dropna(subset=['fechanacimiento'])

In [ ]:
postulantesGenero.head()

In [ ]:
postulantesGenero[postulantesGenero['idpostulante'] == 'PmqM3oj']  #caso raro

In [3]:
postulantesGeneroOHC = pd.concat([postulantesGenero,pd.get_dummies(postulantesGenero['sexo'], prefix='sexo')],axis=1)

In [4]:
postulantesGeneroOHC = postulantesGeneroOHC[postulantesGeneroOHC['sexo_0.0'] == 0]
postulantesGeneroOHC = postulantesGeneroOHC[['idpostulante','fechanacimiento','sexo_FEM','sexo_MASC','sexo_NO_DECLARA']]

In [ ]:
postulantesGeneroOHC.head()

## Postulantes Educacion

In [5]:
postulantesEducacionOHC = pd.concat([postulantesEducacion,pd.get_dummies(postulantesEducacion['estado'], prefix='estado')],axis=1)
postulantesEducacionOHC = pd.concat([postulantesEducacionOHC,pd.get_dummies(postulantesEducacion['nombre'], prefix='nombre')],axis=1)

In [6]:
postulantesEducacionOHC.drop(['nombre'],axis=1, inplace=True)
postulantesEducacionOHC.drop(['estado'],axis=1, inplace=True)

In [ ]:
postulantesEducacionOHC.head()

## Avisos Detalle

In [ ]:
avisosDetalle.head()

In [7]:
#Para losrimeros casos, me voy a quedar con la zona, tipo de trabajo, nivel laboral y nombre de area
avisosDetalleFiltered = avisosDetalle[['idaviso','nombre_zona','tipo_de_trabajo','nivel_laboral','nombre_area']]

In [8]:
avisosDetalleFiltered = avisosDetalleFiltered.loc[avisosDetalleFiltered['nombre_zona'].notnull(),:]
avisosDetalleFiltered = avisosDetalleFiltered.loc[avisosDetalleFiltered['nombre_area'].notnull(),:]
avisosDetalleFiltered = avisosDetalleFiltered.loc[avisosDetalleFiltered['tipo_de_trabajo'].notnull(),:]
avisosDetalleFiltered = avisosDetalleFiltered.loc[avisosDetalleFiltered['nivel_laboral'].notnull(),:]

In [ ]:
#avisosDetalleFiltered['nombre_area'].unique()
#avisosDetalleFiltered['nombre_zona'].unique()
#avisosDetalleFiltered['nivel_laboral'].unique()
avisosDetalleFiltered['tipo_de_trabajo'].unique()

In [ ]:
avisosDetalleFiltered.head()

In [9]:
avisosDetalleOHC = pd.concat([avisosDetalleFiltered,pd.get_dummies(avisosDetalleFiltered['nombre_zona'], prefix='zona')],axis=1)
avisosDetalleOHC = pd.concat([avisosDetalleOHC,pd.get_dummies(avisosDetalleFiltered['tipo_de_trabajo'], prefix='tipo')],axis=1)
avisosDetalleOHC = pd.concat([avisosDetalleOHC,pd.get_dummies(avisosDetalleFiltered['nivel_laboral'], prefix='nivel')],axis=1)

In [10]:
avisosDetalleOHC.drop(['nombre_zona'],axis=1, inplace=True)
avisosDetalleOHC.drop(['tipo_de_trabajo'],axis=1, inplace=True)
avisosDetalleOHC.drop(['nivel_laboral'],axis=1, inplace=True)

avisosDetalleOHC.drop(['nombre_area'],axis=1, inplace=True)

In [ ]:
avisosDetalleOHC.head()

In [11]:
postulacionesFiltered = postulaciones[['idpostulante','idaviso']]
postulacionesFiltered.count()

idpostulante    4909641
idaviso         4909641
dtype: int64

In [12]:
postulacionesFiltered =  postulacionesFiltered.drop_duplicates()
postulacionesFiltered.count()

idpostulante    4909025
idaviso         4909025
dtype: int64

In [13]:
todoMergeado = pd.merge(postulacionesFiltered,avisosDetalleOHC,how='inner',left_on='idaviso',right_on='idaviso')

In [14]:
todoMergeado = pd.merge(todoMergeado,postulantesEducacionOHC, how='inner', left_on='idpostulante',right_on='idpostulante')

In [15]:
todoMergeado = pd.merge(todoMergeado,postulantesGeneroOHC, how='inner', left_on='idpostulante',right_on='idpostulante')

In [17]:
todoMergeado['sepostulo'] = 1

In [18]:
todoMergeado.drop_duplicates() 
#CHEQUEAR DUPLICADOS...

,idpostulante,idaviso,zona_Capital Federal,zona_GBA Oeste,zona_Gran Buenos Aires,tipo_Fines de Semana,tipo_Full-time,tipo_Part-time,tipo_Pasantia,tipo_Por Contrato,...,nombre_Otro,nombre_Posgrado,nombre_Secundario,nombre_Terciario/Técnico,nombre_Universitario,fechanacimiento,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,sepostulo
0,NjlD,1112248724,0,0,1,0,1,0,0,0,...,0,0,0,0,1,1981-06-02,1,0,0,1
1,NjlD,1112248724,0,0,1,0,1,0,0,0,...,0,0,0,0,1,1981-06-02,1,0,0,1
2,MV6eM8N,1112248724,0,0,1,0,1,0,0,0,...,0,0,0,0,1,1991-10-11,0,1,0,1
3,MV6eM8N,1112248724,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1991-10-11,0,1,0,1
4,MV6eM8N,1112304156,0,0,1,0,1,0,0,0,...,0,0,0,0,1,1991-10-11,0,1,0,1
5,MV6eM8N,1112304156,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1991-10-11,0,1,0,1
6,MV6eM8N,1112445502,0,0,1,0,1,0,0,0,...,0,0,0,0,1,1991-10-11,0,1,0,1
7,MV6eM8N,1112445502,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1991-10-11,0,1,0,1
8,MV6eM8N,1112254843,1,0,0,0,1,0,0,0,...,0,0,0,0,1,1991-10-11,0,1,0,1
9,MV6eM8N,1112254843,1,0,0,0,1,0,0,0,...,0,0,0,1,0,1991-10-11,0,1,0,1


In [19]:
#Creo un sample
todoMergeadoSample = todoMergeado.sample(frac=0.01)

In [20]:
##Avisos para los que no se postulo



##avisosNoPostulado = pd.merge(postulaciones,todoMergeado,on=['idpostulante', 'idaviso'] ,how='outer')
avisosNoPostulado = pd.merge(postulaciones,todoMergeadoSample,on=['idpostulante', 'idaviso'] ,how='outer')

In [21]:
avisosNoPostulado.head()

,idaviso,idpostulante,fechapostulacion,zona_Capital Federal,zona_GBA Oeste,zona_Gran Buenos Aires,tipo_Fines de Semana,tipo_Full-time,tipo_Part-time,tipo_Pasantia,...,nombre_Otro,nombre_Posgrado,nombre_Secundario,nombre_Terciario/Técnico,nombre_Universitario,fechanacimiento,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,sepostulo
0,1112248724,NjlD,2018-01-19 07:39:16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,1112286523,ZaO5,2018-01-24 15:07:39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2,1112272060,ZaO5,2018-01-24 15:20:10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
3,1112288401,ZaO5,2018-01-26 08:37:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
4,1112300563,ZaO5,2018-01-30 13:35:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [22]:
avisosNoPostulado['sepostulo'] = 0

In [ ]:
avisosNoPostulado.head()

In [ ]:
todoMergeado.head()

In [23]:
#todoMergeadoFINAL = pd.concat(avisosNoPostulado,todoMergeado, ignore_index=True)
todoMergeadoFINAL = pd.concat([avisosNoPostulado, todoMergeadoSample], axis=0, ignore_index=True)

In [ ]:
todoMergeadoFINAL.head()

In [24]:
todoMergeadoFINAL.info

<bound method DataFrame.info of          estado_Abandonado  estado_En Curso  estado_Graduado fechanacimiento  \
0                      NaN              NaN              NaN             NaT   
1                      NaN              NaN              NaN             NaT   
2                      NaN              NaN              NaN             NaT   
3                      NaN              NaN              NaN             NaT   
4                      NaN              NaN              NaN             NaT   
5                      NaN              NaN              NaN             NaT   
6                      NaN              NaN              NaN             NaT   
7                      NaN              NaN              NaN             NaT   
8                      NaN              NaN              NaN             NaT   
9                      NaN              NaN              NaN             NaT   
10                     NaN              NaN              NaN             NaT   
11      

In [26]:
#lo separo porque sino explota todo
datosForTraining = todoMergeadoFINAL

In [79]:
datosForTraining.head()

,estado_Abandonado,estado_En Curso,estado_Graduado,fechanacimiento,fechapostulacion,idaviso,idpostulante,nivel_Gerencia / Alta Gerencia / Dirección,nivel_Jefe / Supervisor / Responsable,nivel_Junior,...,tipo_Pasantia,tipo_Por Contrato,tipo_Por Horas,tipo_Primer empleo,tipo_Teletrabajo,tipo_Temporario,tipo_Voluntario,zona_Capital Federal,zona_GBA Oeste,zona_Gran Buenos Aires
0,NaN,NaN,NaN,NaT,2018-01-19 07:39:16,1112248724,NjlD,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,1.0,1978-08-10,2018-01-24 15:07:39,1112286523,ZaO5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,NaN,NaN,NaN,NaT,2018-01-24 15:20:10,1112272060,ZaO5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaT,2018-01-26 08:37:04,1112288401,ZaO5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaT,2018-01-30 13:35:48,1112300563,ZaO5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
datosForTrainingNumericals = datosForTraining.drop(['fechanacimiento','fechapostulacion'],axis=1)

In [30]:
datosForTrainingNumericals.dropna(axis=0,inplace=True, how='any')

In [31]:
datosForTrainingNumericals.set_index('idpostulante','idaviso')

,estado_Abandonado,estado_En Curso,estado_Graduado,idaviso,nivel_Gerencia / Alta Gerencia / Dirección,nivel_Jefe / Supervisor / Responsable,nivel_Junior,nivel_Otro,nivel_Senior / Semi-Senior,nombre_Doctorado,...,tipo_Pasantia,tipo_Por Contrato,tipo_Por Horas,tipo_Primer empleo,tipo_Teletrabajo,tipo_Temporario,tipo_Voluntario,zona_Capital Federal,zona_GBA Oeste,zona_Gran Buenos Aires
idpostulante,,,,,,,,,,,,,,,,,,,,,
ZaO5,1.0,0.0,0.0,1112420942,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
ZaO5,1.0,0.0,0.0,1112438899,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Ez8J,1.0,0.0,0.0,1112249031,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
erL9,1.0,0.0,0.0,1112425374,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
N0V2D,0.0,0.0,1.0,1110666060,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
N0V2D,0.0,0.0,1.0,1112402322,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Y0Gmvr,0.0,0.0,1.0,1112326665,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Y0Gmvr,0.0,0.0,1.0,1112431273,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
ZaaBeR,0.0,1.0,0.0,1112433224,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [32]:
datosForTrainingNumericals = datosForTrainingNumericals.reset_index()

In [33]:
datosForTrainingNumericals.info

<bound method DataFrame.info of           index  estado_Abandonado  estado_En Curso  estado_Graduado  \
0            52                1.0              0.0              0.0   
1            75                1.0              0.0              0.0   
2           133                1.0              0.0              0.0   
3           149                1.0              0.0              0.0   
4           422                0.0              0.0              1.0   
5           441                0.0              0.0              1.0   
6           556                0.0              0.0              1.0   
7           574                0.0              0.0              1.0   
8           905                0.0              1.0              0.0   
9          1002                0.0              0.0              1.0   
10         1386                0.0              0.0              1.0   
11         1439                0.0              1.0              0.0   
12         1620                0

In [34]:
datosForTrainingNumericals = datosForTrainingNumericals.drop('index',axis=1)

In [35]:
datosForTrainingNumericals = datosForTrainingNumericals.drop('idaviso',axis=1)

In [36]:
datosForTrainingNumericalsSinPostulacion  = datosForTrainingNumericals.drop('sepostulo',axis=1)

In [37]:
datosForTrainingNumericals = datosForTrainingNumericals.drop('idpostulante',axis=1)

In [38]:
datosForTrainingNumericalsSinPostulacion = datosForTrainingNumericalsSinPostulacion.drop('idpostulante',axis=1)

# Entrenamos Machine Learning

In [39]:
# Load libraries
import pandas
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor

In [40]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(datosForTrainingNumericalsSinPostulacion, datosForTrainingNumericals['sepostulo'], test_size=0.2, random_state=7)

In [42]:
models = []
models.append(('LR', LogisticRegression()))
#models.append(('LDA', LinearDiscriminantAnalysis()))
#models.append(('KNN', KNeighborsClassifier()))
#models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
#models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=7)
    cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

LR: 0.491642 (0.003106)
NB: 0.496065 (0.002800)


In [ ]:
from sklearn.metrics import r2_score

models = []
#models.append(('LR', LogisticRegression()))
#models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
#models.append(('CART', DecisionTreeClassifier()))
#models.append(('RF', RandomForestRegressor(n_estimators=100, max_features='sqrt')))
#models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []
for name, model in models:
    model.fit(X_train, Y_train)
    cv_results = r2_score(Y_validation, model.predict(Y_train))
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

# Prueba 1

In [ ]:
testingSet = pd.read_csv('resources/test_final_100k.csv')

In [216]:
#testingSet.head()

testingSetMerged = pd.merge(testingSet,postulantesEducacionOHC,how='left',on='idpostulante')
testingSetMerged = pd.merge(testingSetMerged,postulantesGeneroOHC,how='left',on='idpostulante')


In [221]:
testingSetMerged = pd.merge(testingSetMerged,avisosDetalleOHC,how='left',on='idaviso')

In [222]:
#todoMergeado.columns
testingSetMerged = testingSetMerged.loc[testingSetMerged['id'].notnull(),:]

In [223]:
testingSetMerged = testingSetMerged.drop_duplicates(subset='id')

In [225]:
testingSetMerged.info

<bound method DataFrame.info of            id     idaviso idpostulante  estado_Abandonado  estado_En Curso  \
0           0      739260       6M9ZQR                0.0              0.0   
2           1      739260       6v1xdL                0.0              1.0   
5           2      739260       ezRKm9                0.0              0.0   
6           3      758580       1Q35ej                0.0              0.0   
8           4      758580       EAN4J6                0.0              1.0   
11          5      758580       8R6pzR                0.0              0.0   
13          6      776420       aZJ2XN                0.0              0.0   
14          7      776420       Nmpo3J                0.0              1.0   
16          8      776420       eVqWar                0.0              0.0   
17          9      820850       6ZBD33                0.0              1.0   
19         10      820850       1j4RNj                0.0              0.0   
20         11      820850       

In [226]:
testingSetMerged['test_set'] =1

In [227]:
datosForTrainingNumericals['train_set'] = 1

In [228]:
concatenados = pd.concat([testingSetMerged,datosForTrainingNumericals],axis=1)

In [229]:
setDatosToPredict = concatenados[concatenados['test_set'] == 1]

In [230]:
setDatosToPredict.info

<bound method DataFrame.info of              id       idaviso idpostulante  estado_Abandonado  \
0           0.0  7.392600e+05       6M9ZQR                0.0   
2           1.0  7.392600e+05       6v1xdL                0.0   
5           2.0  7.392600e+05       ezRKm9                0.0   
6           3.0  7.585800e+05       1Q35ej                0.0   
8           4.0  7.585800e+05       EAN4J6                0.0   
11          5.0  7.585800e+05       8R6pzR                0.0   
13          6.0  7.764200e+05       aZJ2XN                0.0   
14          7.0  7.764200e+05       Nmpo3J                0.0   
16          8.0  7.764200e+05       eVqWar                0.0   
17          9.0  8.208500e+05       6ZBD33                0.0   
19         10.0  8.208500e+05       1j4RNj                0.0   
20         11.0  8.208500e+05        ZOwE8                0.0   
22         12.0  8.208500e+05       NG4BaD                0.0   
23         13.0  9.148800e+05       6q6eNl                

In [231]:
setToTrain = concatenados[concatenados['train_set'] == 1]

In [233]:
setToTrain = setToTrain.drop(['fechapostulacion','fechanacimiento','idaviso','idpostulante','id','test_set','train_set'],axis=1)

In [234]:
setToTrain.dropna(axis=0,inplace=True, how='any')

In [235]:
setToTrainNumericals = setToTrain.drop('sepostulo',axis=1)

In [236]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(setToTrainNumericals, setToTrain['sepostulo'], test_size=0.2, random_state=7)

In [237]:
g = GaussianNB()
g.fit(X_train, Y_train)

GaussianNB(priors=None)

In [238]:
limpiado = setDatosToPredict.drop(['fechapostulacion','fechanacimiento','idaviso','idpostulante','id','test_set','train_set'],axis=1)

In [240]:
limpiado = limpiado.drop('sepostulo',axis=1)

In [241]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)

In [242]:
prueba1 =g.predict(imp.fit_transform(limpiado))

In [243]:
archivo1 = pd.DataFrame()
archivo1['id'] = testingSet.loc[:,['id']]
archivo1['sepostulo'] = prueba1
archivo1.to_csv('prueba1.csv', index=False)

In [100]:
testingSet.size

300000